# New York City H3 Cells

## Packages and Utility Functions

In [ ]:
from IPython.display import display
from math import log10
from toolz import compose_left as compose, do, juxt
import h3
import numpy as np
import pandas as pd
import shapely
from shapely.geometry import Polygon, mapping, MultiPolygon
import geopandas as gpd
import h3pandas
import folium
import fiona

def load_geocsv(filepath_or_buffer, index_col, geo_col):
    df = pd.read_csv(filepath_or_buffer,
                     index_col=index_col,
                     converters={geo_col: shapely.wkt.loads},
                     dtype='object') \
           .rename(columns={geo_col: 'geometry'})
    return gpd.GeoDataFrame(df)

## Loading Data

In [ ]:
borough_areas = load_geocsv('https://data.cityofnewyork.us/api/views/jbrz-qt9e/rows.csv?accessType=DOWNLOAD', 'BoroCode', 'the_geom')
borough_land = load_geocsv('https://data.cityofnewyork.us/api/views/7t3b-ywvw/rows.csv?accessType=DOWNLOAD', 'BoroCode', 'the_geom')
nta_areas = load_geocsv('https://data.cityofnewyork.us/resource/9nt8-h7nd.csv', 'nta2020', 'the_geom')

## Deriving Additional `GeoDataFrame`s and Visualizing H3 Cells of New York City

### Deriving Additional `GeoataFrame`s

`nyc_area` is used to test whether points or polygons fall within New York City’s boundaries. This includes water. `nyc_land`, however, is used to test whether points or polygons falll within a land mass in New York City. It’s used to exclude points and polygons that are partly or entirely in water.

`h3_polygons` is used to uniformly group areas within New York City. Each polygon is uniform in size. However, they may cross state, city, borough, or neighborhood boundaries.

In [ ]:
nyc_area, nyc_land = (gdf.get(['geometry']) \
                         .dissolve() \
                         .explode(index_parts=False) \
                         .exterior \
                         .apply(Polygon) \
                         .agg(compose(gpd.GeoSeries.to_list, MultiPolygon))
                      for gdf in (borough_areas, borough_land))
h3_polygons = gpd.GeoDataFrame.from_records(({'h3_index': h3.string_to_h3(h3_index),
                                              'only_water': polygon.disjoint(nyc_land),
                                              'geometry': polygon}
                                             for h3_index in h3.h3_to_children(h3.geo_to_h3(*reversed(nyc_area.centroid.coords[0]), 3), 9)
                                             if (polygon := Polygon(h3.h3_to_geo_boundary(h3_index, True))).intersects(nyc_area)),
                                            'h3_index')

#### Areas Within Boroughs But Not Any Neighborhoods—Water

Non-NTA areas within boroughs require special handling. Every part of the borough must be completely subdividable into a areas identified by an `nta2020` code. Since the river and ocean are not a part of any neighborhoods, we assign those areas a two-letter code representing the borough. Deriving the geometry from the shapes is tricky, however. Merely subtracting the boroughs land boundaries from the borough areas will yield imperfect polygons pocked with microscopic holes and mismatched boundaries where different boroughs interface.

As seen below, subtracting the entire shape of New York City’s shoreline yields much better results, though both Ellis Island and Liberty Island exhibit obvious artifacts from the operation and less obvious at the land border separating the Bronx and Queens from Westchester and Long Island respectively.

In [ ]:
borough_water = borough_areas.assign(geometry=borough_areas.geometry.difference(nyc_land)) \
                             .reset_index() \
                             .get(['BoroCode', 'BoroName', 'geometry']) \
                             .explode(ignore_index=True) \
                             .pipe(lambda df: df.assign(area=df.geometry.area)) #\
                             #.query('area >= 10e-11') \
                             #.dissolve('BoroCode')

display(borough_water.sort_values('area'))

do(juxt(folium.Choropleth(geo_data=borough_water.to_json(),
                          name='Water',
                          fill_color='#0080ff',
                          line_color='orange',
                          line_opacity=1) \
              .add_to,
        lambda map: [folium.CircleMarker(location := attributes.geometry.representative_point().coords[0][::-1],
                                         tooltip=f'<strong>{attributes.BoroName}</strong><br />'
                                                 f'Location: {location[0]:.6f}, {location[1]:.6f}<br />'
                                                 f'Log(area): {log10(attributes.area):,.2f}',
                                         radius=max(3, abs(log10(attributes.area))),
                                         color='green' if attributes.area >= 10e-6 else 'orange' if attributes.area >= 10e-12 else 'red',
                                         fill=True,
                                         fill_opacity=1,
                                         stroke=False)
                           .add_to(map)
                     for boro_id, attributes in borough_water.sort_values('area').iterrows()]),
   folium.Map(location=nyc_area.representative_point().coords[0][::-1],
              tiles='Stamen Toner',
              zoom_start=11))

Filtering out the polygons with areas less than 10⁻¹¹ will leave us with only the ones that matter. And those can be added to the `nta_areas` data frame.

In [ ]:
nta_areas = borough_water.query('area >= 10e-11') \
                             .dissolve('BoroCode') \
                             .get(['BoroName', 'geometry']) \
                             .join(pd.DataFrame.from_dict({1: 'MN', 2: 'BX', 3: 'BK', 4: 'QN', 5: 'SI'}, orient='index', columns=['nta2020'])) \
                             .reset_index() \
                             .set_index('nta2020') \
                             .rename(columns={'BoroCode': 'borocode', 'BoroName': 'boroname'}) \
                             .pipe(nta_areas.append)

With the `nta_areas` data frame complete, the relationships between the H3 cells and the neighborhoods can be defined.neighborhoods-to-H3 cells is many-to-many.

In [ ]:
nta2020_h3_relationships = h3_polygons.reset_index() \
                                      .sjoin(borough_areas, how='left', lsuffix='h3', rsuffix='boro') \
                                      .sjoin(nta_areas, how='left', lsuffix='', rsuffix='nta') \
                                      .pipe(lambda df: df.assign(borocode=np.where(df.borocode.isna(), df.index_boro, df.borocode))) \
                                      .join(pd.DataFrame.from_dict({1: 'MN', 2: 'BX', 3: 'BK', 4: 'QN', 5: 'SI'}, orient='index', columns=['boro_abbrev']), 'borocode', 'left') \
                                      .pipe(lambda df: df.assign(index_nta=np.where(df.index_nta.isna(), df.boro_abbrev, df.index_nta))) \
                                      .get(['h3_index', 'index_nta']) \
                                      .drop_duplicates() \
                                      .rename(columns={'index_nta': 'nta2020_id'})

### Visualization

In [ ]:
do(juxt(folium.Choropleth(geo_data=borough_areas.to_json(),
                          name='Borough Boundaries',
                          fill_color='none',
                          line_color='black',
                          line_opacity=0.5) \
              .add_to,
        folium.Choropleth(geo_data=h3_polygons.to_json(),
                          name='H3 Tiles',
                          fill_color='gray',
                          fill_opacity=0.5,
                          line_color='#808080',
                          line_opacity=0.5) \
              .add_to,
        lambda map: [folium.CircleMarker(location := attributes.geometry.representative_point().coords[0][::-1],
                                         tooltip=f'<strong>{attributes.ntaname if type(attributes.ntaname) is str else "🌊"}</strong>, {attributes.boroname}<br />'
                                                 f'<em>({location[0]:.6}, {location[1]:.6})</em><br />'
                                                 f'{len(nta2020_h3_relationships.get(nta2020_h3_relationships.nta2020_id == nta2020_id)):,} hexagon(s)',
                                         radius=3,
                                         color='black',
                                         fill=True,
                                         fill_opacity=0.75,
                                         stroke=False)
                           .add_to(map)
                     for nta2020_id, attributes in nta_areas.iterrows()]),
   folium.Map(location=nyc_area.representative_point().coords[0][::-1],
              tiles='OpenStreetMap',
              zoom_start=11))

## Saving the Geography Data

Geography data is used by two functions in this project.

1. When data is ingested periodically from NYC OpenData, individual collisions are tagged with borough, neighborhood, and H3 identifiers/indices. The tagging shifts the expensive geospatial joins to the relatively infrequent ingestion process to enable inexpensive queries.
2. When requested, the API supplies geometry for the requested borough, neighborhood, or H3 index.

### New York City Boundaries

This includes all of the areas belonging to New York City including water.

In [ ]:
with fiona.open('nyc_area.gpkg', 'w', 'GPKG', {'geometry': 'MultiPolygon'}) as file:
    file.write({'geometry': mapping(nyc_area)})

### Borough Boundaries

The CSV version has the addition of one more geometry column containing the shape of just the land masses.

In [ ]:
juxt(lambda df: df.join(borough_land.get(['geometry']).rename(columns={'geometry': 'land_geometry'}))
                  .to_csv('boro.csv'),
     lambda df: df.to_file('boro.shp')) \
(borough_areas.reset_index()
              .rename(columns={'BoroCode': 'id',
                               'BoroName': 'name'})
              .get(['id', 'name', 'geometry'])
              .sort_values('id')
              .set_index('id'));

### Neighborhood Boundaries

In [ ]:
juxt(lambda df: df.to_csv('nta2020.csv'),
     lambda df: df.to_file('nta2020.shp')) \
(nta_areas.reset_index()
          .rename(columns={'nta2020': 'id',
                           'borocode': 'boro_code',
                           'ntaname': 'name'})
          .get(['id', 'name', 'boro_code', 'geometry'])
          .sort_values('id')
          .set_index('id'));

### H3 Cells

In [ ]:
juxt(lambda df: df.to_csv('h3.csv'),
     lambda df: df.to_file('h3.shp')) \
(h3_polygons.sort_index());

### Relationships between Neighborhoods and H3 Cells

In [ ]:
nta2020_h3_relationships.sort_index() \
                        .sort_values(['nta2020_id']) \
                        .get(['h3_index', 'nta2020_id']) \
                        .to_csv('h3_nta2020.csv', index=False)